In [3]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta


In [7]:
ticker_list = [
    "RELIANCE.NS", "HDFCBANK.NS", "ICICIBANK.NS", "INFY.NS", "TCS.NS",
    "LT.NS", "ITC.NS", "KOTAKBANK.NS", "HINDUNILVR.NS", "AXISBANK.NS",
    "SBIN.NS", "BHARTIARTL.NS", "BAJFINANCE.NS", "ASIANPAINT.NS", "MARUTI.NS",
    "HCLTECH.NS", "SUNPHARMA.NS", "NTPC.NS", "TITAN.NS", "ONGC.NS",
    "ULTRACEMCO.NS", "POWERGRID.NS", "ADANIENT.NS", "JSWSTEEL.NS", "WIPRO.NS",
    "TATAMOTORS.NS", "INDUSINDBK.NS", "BAJAJFINSV.NS", "HINDALCO.NS", "GRASIM.NS",
    "TECHM.NS", "DIVISLAB.NS", "NESTLEIND.NS", "TATASTEEL.NS", "CIPLA.NS",
    "ADANIPORTS.NS", "DRREDDY.NS", "BPCL.NS", "SBILIFE.NS", "BRITANNIA.NS",
    "EICHERMOT.NS", "HEROMOTOCO.NS", "COALINDIA.NS", "BAJAJ-AUTO.NS", "HDFCLIFE.NS",
    "SHREECEM.NS", "APOLLOHOSP.NS", "M&M.NS", "ICICIPRULI.NS", "UPL.NS"
]



In [12]:
def process_stock(ticker):
    Data=yf.download(tickers=ticker,period="5y",interval="1d",group_by=ticker)
    
    Data.columns = Data.columns.droplevel(0)
    # Data.head(5)
    
    ## Add Useful KPIs
    
    Data['RSI']=ta.rsi(Data["Close"], length=14)
    
    info = yf.Ticker(ticker).info
    Data['Sector']=info.get("sector")
    Data['Industry']=info.get("industry")
    
    MACD=ta.macd(Data["Close"], fast=12, slow=26, signal=9)
    Data=pd.concat([Data,MACD],axis=1)
    
    Data.rename(columns={
        "MACD_12_26_9": "MACD_Line",
        "MACDs_12_26_9": "MACD_Signal",
        "MACDh_12_26_9": "MACD_Histogram"
    }, inplace=True)
    
    
    Data["EMA_20"] = ta.ema(Data["Close"], length=20)
    Data["EMA_50"] = ta.ema(Data["Close"], length=50)
    
    # Add Bollinger Bands (20-day default)
    bbands = ta.bbands(Data["Close"], length=20)
    
    # Combine with main DataFrame
    Data = pd.concat([Data, bbands], axis=1)
    
    # Optional rename (if needed)
    Data.rename(columns={
        "BBL_20_2.0": "BB_Lower",
        "BBM_20_2.0": "BB_Middle",
        "BBU_20_2.0": "BB_Upper",
        "BBB_20_2.0": "BB_Bandwidth",
        "BBP_20_2.0": "BB_Percent"
    }, inplace=True)
    
    # Add ADX (default 14-period)
    adx_df = ta.adx(Data["High"], Data["Low"], Data["Close"], length=14)
    
    # Join it with main DataFrame
    Data = pd.concat([Data, adx_df], axis=1)
    
    # Optional rename (for clarity)
    Data.rename(columns={
        "ADX_14": "ADX",
        "DMP_14": "DI_Plus",
        "DMN_14": "DI_Minus"
    }, inplace=True)
    
    # Add ATR (default length=14)
    Data["ATR_14"] = ta.atr(Data["High"], Data["Low"], Data["Close"], length=14)
    
    # Add Stochastic Oscillator (default k=14, d=3)
    stoch_df = ta.stoch(Data["High"], Data["Low"], Data["Close"], k=14, d=3)
    
    # Combine with main DataFrame
    Data = pd.concat([Data, stoch_df], axis=1)
    
    # Optional rename
    Data.rename(columns={
        "STOCHk_14_3_3": "Stoch_%K",
        "STOCHd_14_3_3": "Stoch_%D"
    }, inplace=True)
    
    
    
    # Add CCI (default period = 20)
    Data["CCI_20"] = ta.cci(Data["High"], Data["Low"], Data["Close"], length=20)
    
    # Data.to_csv("test.csv")
    
    Data.reset_index(inplace=True)
    
    ## Feature Engineering
    
    Data['Volume1']=Data['Volume'].shift(1)
    Data['Volume_Inc']=(Data['Volume']-Data['Volume1'])/Data['Volume1']
    
    Data['EMA_20_Price']=(Data['Close']-Data['EMA_20'])/Data['EMA_20']
    Data['EMA_50_Price']=(Data['Close']-Data['EMA_50'])/Data['EMA_50']
    
    Data['Stoch_k_D']=(Data['Stoch_%K']-Data['Stoch_%D'])
    
    Data['Close1']=Data['Close'].shift(10)
    Data['Close_Inc']=(Data['Close1']-Data['Close'])/Data['Close']
    
    # Data.head(50)
    
    def Buy_Variable(row):
        if row['Close_Inc']>=0.03:
            return 1
        else:
            return 0
    
    Data['Buy']=Data.apply(Buy_Variable,axis=1)
    
    ## Data Clean
    
    Clean_Data=Data[['RSI', 'MACD_Histogram','BB_Percent','ADX','ATR_14', 'Stoch_%K',
           'Stoch_%D', 'CCI_20','Volume_Inc', 'EMA_20_Price',
           'EMA_50_Price', 'Stoch_k_D', 'Buy']]
    
    Clean_Data=Clean_Data.dropna(subset=['RSI', 'MACD_Histogram','BB_Percent','ADX','ATR_14', 'Stoch_%K',
           'Stoch_%D', 'CCI_20','Volume_Inc', 'EMA_20_Price',
           'EMA_50_Price'])
    
    # Clean_Data.head(10)
    
    ## Visualize
    
    import seaborn as sns
    import matplotlib.pyplot as plt
    
    features = ["RSI", "MACD_Histogram", "BB_Percent", "ADX", "ATR_14", 
                "Stoch_%K", "CCI_20", "EMA_20_Price", "Volume_Inc"]
    
    # for feature in features:
    #     plt.figure(figsize=(6, 3))
    #     sns.kdeplot(data=Clean_Data[Clean_Data["Buy"] == 1], x=feature, label="Buy = 1", shade=True)
    #     sns.kdeplot(data=Clean_Data[Clean_Data["Buy"] == 0], x=feature, label="Buy = 0", shade=True)
    #     plt.title(f"{feature} vs Buy")
    #     plt.legend()
    #     plt.tight_layout()
    #     plt.show()
    
    
    # for feature in features:
    #     plt.figure(figsize=(6, 3))
    #     sns.boxplot(data=Clean_Data, x="Buy", y=feature)
    #     plt.title(f"{feature} by Buy value")
    #     plt.tight_layout()
    #     plt.show()
    
    
    Clean_Data.drop(columns="Volume_Inc",inplace=True)
    
    # # Step 1: Compute correlation matrix
    # corr_matrix = Clean_Data.corr()
    
    # # Step 2: Set up the heatmap
    # plt.figure(figsize=(12, 8))
    # sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", linewidths=0.5)
    # plt.title("Correlation Heatmap")
    # plt.tight_layout()
    # plt.show()
    
    # Clean_Data.head(5)
    
    ## Model Build
    
    from sklearn.preprocessing import StandardScaler
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(Clean_Data.drop("Buy", axis=1))
    
    
    
    # X_scaled
    
    from sklearn.model_selection import train_test_split
    y=Clean_Data['Buy']
    
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # ### Logisitic
    
    # from sklearn.linear_model import LogisticRegression
    
    # model=LogisticRegression()
    # model.fit(X_train, y_train)
    # y=model.predict(X_test)
    # y_proba = model.predict_proba(X_test)[:, 1]
    # from sklearn.metrics import classification_report,roc_auc_score
    # print("🔹 Logistic Regression:")
    # print(classification_report(y_test, y))
    # print(f"ROC AUC Score: {roc_auc_score(y_test, y_proba):.4f}")
    
    # ### Random Forest
    
    # from sklearn.ensemble import RandomForestClassifier
    
    # model=RandomForestClassifier(random_state=42)
    # model.fit(X_train, y_train)
    # y=model.predict(X_test)
    # y_proba = model.predict_proba(X_test)[:, 1]
    # feature_importances = model.feature_importances_
    # features = Clean_Data.drop("Buy", axis=1).columns
    # # Combine into a DataFrame
    # F_Imp = pd.DataFrame({
    #     "Feature": features,
    #     "Importance": feature_importances
    # }).sort_values("Importance", ascending=False)
    # print(F_Imp)
    # from sklearn.metrics import classification_report
    # print("🔹 RandomForestClassifier:")
    # print(classification_report(y_test, y))
    # print(f"ROC AUC Score: {roc_auc_score(y_test, y_proba):.4f}")
    
    ### XG Boost
    
    from xgboost import XGBClassifier
    model=XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
    model.fit(X_train, y_train)
    y=model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]
    feature_importances = model.feature_importances_
    features = Clean_Data.drop("Buy", axis=1).columns
    # Combine into a DataFrame
    F_Imp = pd.DataFrame({
        "Feature": features,
        "Importance": feature_importances
    }).sort_values("Importance", ascending=False)
    # print(F_Imp)
    from sklearn.metrics import classification_report
    # print("🔹 XGBClassifier: ")
    # print(classification_report(y_test, y))
    # print(f"ROC AUC Score: {roc_auc_score(y_test, y_proba):.4f}")
    
    
    
    data_Test = yf.download(tickers=ticker, period="60d", interval="1d", group_by="ticker")
    data_Test.columns = data_Test.columns.droplevel(0)
    
    data_Test['RSI']=ta.rsi(data_Test["Close"], length=14)
    
    info = yf.Ticker(ticker).info
    data_Test['Sector']=info.get("sector")
    data_Test['Industry']=info.get("industry")
    
    MACD=ta.macd(data_Test["Close"], fast=12, slow=26, signal=9)
    data_Test=pd.concat([data_Test,MACD],axis=1)
    
    data_Test.rename(columns={
        "MACD_12_26_9": "MACD_Line",
        "MACDs_12_26_9": "MACD_Signal",
        "MACDh_12_26_9": "MACD_Histogram"
    }, inplace=True)
    
    
    data_Test["EMA_20"] = ta.ema(data_Test["Close"], length=20)
    data_Test["EMA_50"] = ta.ema(data_Test["Close"], length=50)
    
    # Add Bollinger Bands (20-day default)
    bbands = ta.bbands(data_Test["Close"], length=20)
    
    # Combine with main DataFrame
    data_Test = pd.concat([data_Test, bbands], axis=1)
    
    # Optional rename (if needed)
    data_Test.rename(columns={
        "BBL_20_2.0": "BB_Lower",
        "BBM_20_2.0": "BB_Middle",
        "BBU_20_2.0": "BB_Upper",
        "BBB_20_2.0": "BB_Bandwidth",
        "BBP_20_2.0": "BB_Percent"
    }, inplace=True)
    
    # Add ADX (default 14-period)
    adx_df = ta.adx(data_Test["High"], data_Test["Low"], data_Test["Close"], length=14)
    
    # Join it with main DataFrame
    data_Test = pd.concat([data_Test, adx_df], axis=1)
    
    # Optional rename (for clarity)
    data_Test.rename(columns={
        "ADX_14": "ADX",
        "DMP_14": "DI_Plus",
        "DMN_14": "DI_Minus"
    }, inplace=True)
    
    # Add ATR (default length=14)
    data_Test["ATR_14"] = ta.atr(data_Test["High"], data_Test["Low"], data_Test["Close"], length=14)
    
    # Add Stochastic Oscillator (default k=14, d=3)
    stoch_df = ta.stoch(data_Test["High"], data_Test["Low"], data_Test["Close"], k=14, d=3)
    
    # Combine with main DataFrame
    data_Test = pd.concat([data_Test, stoch_df], axis=1)
    
    # Optional rename
    data_Test.rename(columns={
        "STOCHk_14_3_3": "Stoch_%K",
        "STOCHd_14_3_3": "Stoch_%D"
    }, inplace=True)
    
    
    
    # Add CCI (default period = 20)
    data_Test["CCI_20"] = ta.cci(data_Test["High"], data_Test["Low"], data_Test["Close"], length=20)
    
    data_Test['Volume1']=data_Test['Volume'].shift(1)
    data_Test['Volume_Inc']=(data_Test['Volume']-data_Test['Volume1'])/data_Test['Volume1']
    
    data_Test['EMA_20_Price']=(data_Test['Close']-data_Test['EMA_20'])/data_Test['EMA_20']
    data_Test['EMA_50_Price']=(data_Test['Close']-data_Test['EMA_50'])/data_Test['EMA_50']
    
    data_Test['Stoch_k_D']=(data_Test['Stoch_%K']-data_Test['Stoch_%D'])
    
    data_Test['Close1']=data_Test['Close'].shift(10)
    data_Test['Close_Inc']=(data_Test['Close1']-data_Test['Close'])/data_Test['Close']
    
    data_Test.head(50)
    
    def Buy_Variable(row):
        if row['Close_Inc']>=0.03:
            return 1
        else:
            return 0
    
    data_Test['Buy']=data_Test.apply(Buy_Variable,axis=1)
    
    data_Test.iloc[-2:-1]
    
    Clean_Data_Test=data_Test[['RSI', 'MACD_Histogram','BB_Percent','ADX','ATR_14', 'Stoch_%K',
           'Stoch_%D', 'CCI_20','Volume_Inc', 'EMA_20_Price',
           'EMA_50_Price', 'Stoch_k_D', 'Buy']]
    
    # Step 1: Get feature columns (same as used during training)
    features = ['RSI', 'MACD_Histogram', 'BB_Percent', 'ADX', 'ATR_14',
                'Stoch_%K', 'Stoch_%D', 'CCI_20', 'EMA_20_Price', 'EMA_50_Price', 'Stoch_k_D']
    
    # Step 2: Extract today's row (last row in the test table)
    today_row = Clean_Data_Test[features].iloc[-2:-1]  # Keep it as DataFrame
    
    # Step 3: Apply the same scaler used during training
    today_scaled = scaler.transform(today_row)
    
    # Step 4: Predict using the trained model
    buy_prediction = model.predict(today_scaled)[0]
    print(today_scaled)
    buy_probability = model.predict_proba(today_scaled)[0][1]
    
    # Step 5: Print result
    # print("🔮 Buy Prediction:", "Yes" if buy_prediction == 1 else "No")
    # print(f"📊 Confidence: {buy_probability:.2%}")
    return {
            "Ticker": ticker,
            "Buy": "Yes" if buy_prediction == 1 else "No",
            "Confidence": round(buy_probability * 100, 2)
        }
    
    


In [13]:
results = []

for ticker in ticker_list:
    result = process_stock(ticker)
    results.append(result)

final_df = pd.DataFrame(results)
print(final_df)


C:\Users\PC\AppData\Local\Temp\ipykernel_2912\182687533.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  Data=yf.download(tickers=ticker,period="5y",interval="1d",group_by=ticker)
[*********************100%***********************]  1 of 1 completed
C:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [01:44:47] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\PC\AppData\Local\Temp\ipykernel_2912\182687533.py:229: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data_Test = yf.download(tickers=ticker, period="60d", interval="1d", group_by="ticker")
[*********************100%***********************]  1 of 1 completed


[[ 0.8973778   0.46007967  0.61649601  1.80056829  0.04929317  1.25550567
   1.43997144  0.66762544  0.50981621  0.9589555  -0.49261063]]


C:\Users\PC\AppData\Local\Temp\ipykernel_2912\182687533.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  Data=yf.download(tickers=ticker,period="5y",interval="1d",group_by=ticker)
[*********************100%***********************]  1 of 1 completed
C:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [01:44:49] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\PC\AppData\Local\Temp\ipykernel_2912\182687533.py:229: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data_Test = yf.download(tickers=ticker, period="60d", interval="1d", group_by="ticker")
[*********************100%***********************]  1 of 1 completed


[[ 1.0288949   0.37057249  0.76692102  0.02949055 -0.58923532  1.06718124
   0.92212808  0.59172778  0.52364047  0.55597959  0.58573061]]


C:\Users\PC\AppData\Local\Temp\ipykernel_2912\182687533.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  Data=yf.download(tickers=ticker,period="5y",interval="1d",group_by=ticker)
[*********************100%***********************]  1 of 1 completed
C:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [01:44:52] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\PC\AppData\Local\Temp\ipykernel_2912\182687533.py:229: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data_Test = yf.download(tickers=ticker, period="60d", interval="1d", group_by="ticker")
[*********************100%***********************]  1 of 1 completed


...

[[ 0.6165954  -0.13612777  0.4257467  -0.52586692  0.70532211  0.6476301
   0.72002783  0.29739241  0.09510218  0.14411558 -0.17479969]]


C:\Users\PC\AppData\Local\Temp\ipykernel_2912\182687533.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  Data=yf.download(tickers=ticker,period="5y",interval="1d",group_by=ticker)
[*********************100%***********************]  1 of 1 completed
C:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [01:47:01] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\PC\AppData\Local\Temp\ipykernel_2912\182687533.py:229: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data_Test = yf.download(tickers=ticker, period="60d", interval="1d", group_by="ticker")
[*********************100%***********************]  1 of 1 completed


[[ 1.2837905   0.83095205  1.6656155  -0.93754151 -0.43446667  1.50839182
   1.14127287  1.45324159  1.14080709  1.04776558  1.5074629 ]]


C:\Users\PC\AppData\Local\Temp\ipykernel_2912\182687533.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  Data=yf.download(tickers=ticker,period="5y",interval="1d",group_by=ticker)
[*********************100%***********************]  1 of 1 completed
C:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [01:47:04] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\PC\AppData\Local\Temp\ipykernel_2912\182687533.py:229: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data_Test = yf.download(tickers=ticker, period="60d", interval="1d", group_by="ticker")
[*********************100%***********************]  1 of 1 completed


[[ 0.41330506  0.71319783  0.45397285 -0.30226885 -0.79612784  0.48379887
   0.79490969  0.50502973  0.15948759  0.09644637 -1.05565873]]
           Ticker  Buy  Confidence
0     RELIANCE.NS   No        0.00
1     HDFCBANK.NS   No        0.01
2    ICICIBANK.NS   No        2.03
3         INFY.NS   No        0.01
4          TCS.NS   No        0.03
5           LT.NS   No       30.39
6          ITC.NS   No        0.02
7    KOTAKBANK.NS   No        0.01
8   HINDUNILVR.NS   No        0.00
9     AXISBANK.NS  Yes       98.67
10        SBIN.NS   No        0.02
11  BHARTIARTL.NS   No        0.00
12  BAJFINANCE.NS   No        0.02
13  ASIANPAINT.NS   No        0.01
14      MARUTI.NS   No        5.28
15     HCLTECH.NS   No        2.06
16   SUNPHARMA.NS   No        0.01
17        NTPC.NS   No        0.00
18       TITAN.NS  Yes       97.92
19        ONGC.NS   No        1.14
20  ULTRACEMCO.NS   No        0.01
21   POWERGRID.NS   No        0.03
22    ADANIENT.NS   No        0.18
23    JSWSTEEL.NS   No